In [36]:
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
import sys
from keras_preprocessing import image
import random

In [37]:
DATA_PATH = 'images/'

In [50]:
p = Path('images/')
dirs = p.glob('*')

image_data = []

for folder_name in dirs:
    
    for image_path in folder_name.glob('*.jpg'):
        
        img = image.load_img(image_path,target_size=(96,96))
        img_array = image.img_to_array(img)
        image_data.append(img_array)

In [52]:
print(img_data.shape)

(0,)


In [34]:
def drawImg(image):
    
    from matplotlib import pyplot as plt
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    return

for ix in range(10):
    drawImg(img_data[idx[ix]])

NameError: name 'idx' is not defined

In [83]:
HEIGHT = 96
WIDTH = 96
DEPTH = 3

SIZE = HEIGHT*WIDTH*DEPTH

In [39]:
p = Path(DATA_PATH)
dirs = p.glob('*')

image_data = []

for folder_name in dirs:

    for image_path in folder_name.glob('*.jpg'):

        img = image.load_img(image_path,target_size=(96,96))
        img_array = image.img_to_array(img)
        image_data.append(img_array)

        img_data = np.array(image_data, dtype='float32')/255.0   ##Normalizing the data

In [43]:
idx = list(range(0,808))
random.shuffle(idx)
imgs = img_data[idx]

IndexError: index 587 is out of bounds for axis 0 with size 0

In [51]:
img_data = np.array(image_data, dtype='float32')/255.0   ##Normalizing the data